In [10]:
!nvidia-smi

No devices were found


In [20]:
import glob
import os
import json
import re

In [21]:
def process_log(file_path):
    qas = []
    with open(file_path, 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    for i in range(len(lines) - 1):
        if lines[i].startswith('Q: '):
            question = lines[i][len('Q: '):].strip()
            question = re.sub(r'True or false:\s*(.+)$', r'Therefore, \1', question)
            q_id = f"{os.path.basename(file_path).split('.')[0]}_line_{i+1}"

            a_line = lines[i + 1]
            if a_line.startswith('A: '):
                answer = a_line.split()[-1] == "True"
                qas.append({
                    'id': q_id,
                    'question': question,
                    'answer': answer
                })
    return qas

In [22]:
directory = "/home/lh3288/project/PJ2504_Dual_stream/LLM_Dual_Stream/LLM_Probing/data/ProntoQA/"
processed_qa = []
pattern = os.path.join(directory, '*.log')

for file_path in glob.glob(pattern):
    processed_qa.extend(process_log(file_path))

with open(directory + "prontoqa.jsonl", 'w') as f:
    for qa in processed_qa:
        f.write(json.dumps(qa) + "\n")

In [23]:
yes_true = yes_false = not_true = not_false = total = 0

for qa in processed_qa:
    question = qa["question"]
    question = question.split("Therefore, ")[-1]
    answer = qa["answer"]

    total += 1
    if "not" in question and answer:
        not_true += 1
    elif "not" in question and not answer:
        not_false += 1
    elif not "not" in question and answer:
        yes_true += 1
    elif not "not" in question and not answer:
        yes_false += 1

print(total, yes_true, yes_false, not_true, not_false)

19200 4692 4828 4794 4886
